In [23]:
import itertools # join list trhougth permutations
import numpy as np  # just in case math is needed
import random # shuffle index tool
import csv # cvs file format managemnent
import os
import spacy

# Loss Runs data base augmentation #

## Dates data ##

In [2]:
# Generate data files
 
# create the days list
days = [str(i) for i in range(1,32)]
 
# create the monts list
monts = [str(i) for i in range(1,13)]
 
# crete the years list
years = [str(1900 + i) for i in range(0,150)]
 
# create the monts list in letters
monts_letter = ['Jan','Feb','Mar','Apr', 'MAY','JUN', 'JUL', 'AUG','sep','oct','nov','dec']
 
# join the monts width the days
days_monts = [day + '/' + mont for day in days for mont in monts] + [mont + '/' + day for day in days for mont in monts]
 
# joind days_monts to years
data_date_diag = [day_mont + '/' + year for day_mont in days_monts for year in years]# + [year + '/' + day for day in days for mont in monts]
 
# change the format diag for dash
data_date_dash = []
 
for i in range(len(data_date_diag)):
  data_date_dash.append(data_date_diag[i].replace('/','-'))
 
# create the days_monts whith letter
 
days_monts_letter = [day + '/' + mont for day in days for mont in monts_letter] + [mont + '/' + day for day in days for mont in monts_letter]
 
data_date_letter_diag = [day_mont + '/' + year for day_mont in days_monts_letter for year in years]
 
data_date_letter_dash = []
 
for i in range(len(data_date_diag)):
  data_date_letter_dash.append(data_date_letter_diag[i].replace('/','-'))
 
# join all formats
 
data_date = data_date_diag + data_date_dash + data_date_letter_diag + data_date_letter_dash

random.shuffle(data_date)
# take formar dd/mm/yy
for i in range(np.int(len(data_date)/2)):
  data_date[i] = data_date[i][:-2]

random.shuffle(data_date)

## Data grouping ##

In [18]:
DATA_PATH = './data/data_base/'
 
_lists2list = lambda List: [item for sublist in List for item in sublist]
 
 
# 
#     DATA DISTRIBUTUION AS.
#      _______________    ___________________     ___________________________
#     |READ CVS FILES| => |TRANSFORM TO LIST|  => |PRE PROC LIST, CLEAN DATA|
#   
#      __________________________    __________
#     |MERGE NAMES AND LAS NAMES| => |SAVE DATA|
 
# read companies list 
with open(DATA_PATH + 'loss_runs_companies.csv', newline= '' ) as f:
  reader = csv.reader(f)
  data_companies = list(reader)
 
# read last names list
with open(DATA_PATH + 'loss_runs_last_name.csv', newline= '' ) as f:
  reader = csv.reader(f)
  data_lastnames = list(reader)
 
# read last names
with open(DATA_PATH + 'loss_runs_names.csv', newline= '' ) as f:
  reader = csv.reader(f)
  data_names = list(reader)
 
# list of list to list
 
# compnanies
data_companies_list = _lists2list(data_companies)
# last names
data_lastnames_list = _lists2list(data_lastnames)
# names
data_names_list = _lists2list(data_names)
 
 
# join names and last names lists
 
data_names = [name + ' ' + last for name in data_names_list for last in data_lastnames_list]
random.shuffle(data_names)

In [19]:
# clean companies names
import string 

printable= set(string.printable)

#sent = data_companies_list[696]

for i in range(len(data_companies_list)):
   data_companies_list[i] = ''.join(filter(lambda x: x in printable, data_companies_list[i]))

data_companies = data_companies_list

In [21]:
# random insert abbreviation last names (i.e., Alexander P.)

random.shuffle(data_date)

TRAIN_DATA = []

for i in range(500):
  #print(data_date[i])
  TRAIN_DATA.append((data_date[i], {"entities":[(0,len(data_date[i]),'DATE')]}))

for i in range(len(data_companies)):
  #print(data_date[i])
  TRAIN_DATA.append((data_companies[i], {"entities":[(0,len(data_companies[i]),'INSURER')]}))

for i in range(250):
  #print(data_date[i])
  TRAIN_DATA.append((data_names[i], {"entities":[(0,len(data_names[i]),'NAME')]}))

random.shuffle(TRAIN_DATA)

#TRAIN_DATA

# NER entity model re-trainign #

In [25]:
import random
import spacy
from spacy.util import minibatch, compounding
from pathlib import Path


nlp=spacy.load('en_core_web_sm')

# Getting the pipeline component
ner=nlp.get_pipe("ner")

for _, annotations in TRAIN_DATA:
  for ent in annotations.get("entities"):
    ner.add_label(ent[2])

# disable pipe lines
# Disable pipeline components you dont need to change
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]


# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):

  # Training for 30 iterations
  for iteration in range(50):

    # shuufling examples  before every iteration
    random.shuffle(TRAIN_DATA)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
        print("Losses", losses)

05330486893}
Losses {'ner': 3282.6585090953104}
Losses {'ner': 3287.6426441121093}
Losses {'ner': 3303.098527853488}
Losses {'ner': 3314.089562778472}
Losses {'ner': 3323.4676895189277}
Losses {'ner': 3335.2326902198784}
Losses {'ner': 3348.49895507574}
Losses {'ner': 3355.941215937137}
Losses {'ner': 3369.150020544528}
Losses {'ner': 3378.7117205905906}
Losses {'ner': 3391.4121342349044}
Losses {'ner': 3403.7729817676536}
Losses {'ner': 3415.503650610446}
Losses {'ner': 3425.0020047175876}
Losses {'ner': 3429.997302209138}
Losses {'ner': 3437.061444823741}
Losses {'ner': 3453.5576794791214}
Losses {'ner': 3457.712238674163}
Losses {'ner': 3463.162277569174}
Losses {'ner': 3477.516172398924}
Losses {'ner': 3486.626677979826}
Losses {'ner': 3495.1926582801334}
Losses {'ner': 3510.9764345633976}
Losses {'ner': 3517.8804497468464}
Losses {'ner': 3530.4385847795}
Losses {'ner': 3538.283709679841}
Losses {'ner': 3546.344259475469}
Losses {'ner': 3555.286353324651}
Losses {'ner': 3564.792194

## Save the re-trained model ##